In [1]:
from flask import Flask, render_template, request
import plotly
import plotly.graph_objs as go
import pandas as pd
import json
from sqlalchemy import create_engine
from werkzeug.wrappers import Request, Response

import joblib

model = joblib.load('ModelRF')

In [2]:
app = Flask(__name__)

In [3]:
df = pd.read_csv('Clean_KS_after_EDA.csv',low_memory=False)

In [ ]:
def category_plot(
    cat_plot = 'histplot',
    cat_x = 'main_category', cat_y = 'goal_usd',
    estimator = 'count', hue = 'final_status'):
    
    if cat_plot == 'histplot':
        data = []
        for val in df[hue].unique():
            hist = go.Histogram(
                x=df[df[hue]==val][cat_x],
                y=df[df[hue]==val][cat_y],
                histfunc=estimator,
                name=str(val)
            )
            data.append(hist)
        title='Histogram'
    elif cat_plot == 'boxplot':
        data = []

        for val in df[hue].unique():
            box = go.Box(
                x=df[df[hue] == val][cat_x], #series
                y=df[df[hue] == val][cat_y],
                name=str(val)
        )
            data.append(box)
        title='Box'
        
    if cat_plot == 'histplot':
        layout = go.Layout(
            title=title,
            xaxis=dict(title=cat_x),
            yaxis=dict(title=estimator),
            boxmode = 'group'
        )
    else:
        layout = go.Layout(
            title=title,
            xaxis=dict(title=cat_x),
            yaxis=dict(title=cat_y),
            boxmode = 'group'
        )
    result = {'data': data, 'layout': layout}
    graphJSON = json.dumps(result, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON

@app.route('/')
def index():
    plot = category_plot()
    list_plot = [('histplot', 'Histogram'), ('boxplot', 'Box')]
    list_x = [('country', 'country'), ('main_category', 'main_category'), ('Month', 'Month'), ('Launched_Deadline', 'Launched_Deadline')]
    list_y = [('goal_usd', 'Bill'), ('backers_count', 'backers_count')]
    list_est = [('count', 'Count'), ('avg', 'Average'), ('max', 'Max'), ('min', 'Min')]
    list_hue = [('final_status','final_status')]

    return render_template(
        'category.html',
        plot=plot,
        focus_plot='histplot',
        focus_x='main_category',
        focus_estimator='count',
        focus_hue='final_status',
        drop_plot= list_plot,
        drop_x= list_x,
        drop_y= list_y,
        drop_estimator= list_est,
        drop_hue= list_hue)

@app.route('/cat_fn/<nav>')
def cat_fn(nav):

    # saat klik menu navigasi
    if nav == 'True':
        cat_plot = 'histplot'
        cat_x = 'main_category'
        cat_y = 'goal_usd'
        estimator = 'count'
        hue = 'final_status'
        
    else:
        cat_plot = request.args.get('cat_plot')
        cat_x = request.args.get('cat_x')
        cat_y = request.args.get('cat_y')
        estimator = request.args.get('estimator')
        hue = request.args.get('hue')

    if estimator == None:
        estimator = 'count'
    
    if cat_y == None:
        cat_y = 'goal_usd'
        
    list_plot = [('histplot', 'Histogram'), ('boxplot', 'Box')]
    list_x = [('country', 'country'), ('main_category', 'main_category'), ('Month', 'Month'), ('Launched_Deadline', 'Launched_Deadline')]
    list_y = [('goal_usd', 'goal_usd'), ('backers_count', 'backers_count')]
    list_est = [('count', 'Count'), ('avg', 'Average'), ('max', 'Max'), ('min', 'Min')]
    list_hue = [('final_status','final_status')]

    plot = category_plot(cat_plot, cat_x, cat_y, estimator, hue)
    return render_template(
        # file yang akan menjadi response dari API
        'category.html',
        # plot yang akan ditampilkan
        plot=plot,
        # menu yang akan tampil di dropdown 'Jenis Plot'
        focus_plot=cat_plot,
        # menu yang akan muncul di dropdown 'sumbu X'
        focus_x=cat_x,
        focus_y=cat_y,

        # menu yang akan muncul di dropdown 'Estimator'
        focus_estimator=estimator,
        # menu yang akan tampil di dropdown 'Hue'
        focus_hue=hue,
        # list yang akan digunakan looping untuk membuat dropdown 'Jenis Plot'
        drop_plot= list_plot,
        # list yang akan digunakan looping untuk membuat dropdown 'Sumbu X'
        drop_x= list_x,
        # list yang akan digunakan looping untuk membuat dropdown 'Sumbu Y'
        drop_y= list_y,
        # list yang akan digunakan looping untuk membuat dropdown 'Estimator'
        drop_estimator= list_est,
        # list yang akan digunakan looping untuk membuat dropdown 'Hue'
        drop_hue= list_hue
    )

##################
## SCATTER PLOT ##
##################

# scatter plot function
def scatter_plot(cat_x, cat_y, hue):


    data = []

    for val in df[hue].unique():
        scatt = go.Scatter(
            x = df[df[hue] == val][cat_x],
            y = df[df[hue] == val][cat_y],
            mode = 'markers',
            name = str(val)
        )
        data.append(scatt)

    layout = go.Layout(
        title= 'Scatter',
        title_x= 0.5,
        xaxis=dict(title=cat_x),
        yaxis=dict(title=cat_y)
    )

    result = {"data": data, "layout": layout}

    graphJSON = json.dumps(result,cls=plotly.utils.PlotlyJSONEncoder)

    return graphJSON

@app.route('/scatt_fn')
def scatt_fn():
    cat_x = request.args.get('cat_x')
    cat_y = request.args.get('cat_y')
    hue = request.args.get('hue')

    # WAJIB! default value ketika scatter pertama kali dipanggil
    if cat_x == None and cat_y == None and hue == None:
        cat_x = 'main_category'
        cat_y = 'goal_usd'
        hue = 'final_status'

    # Dropdown menu
    list_x = [('backers_count', 'backers_count')]
    list_y = [('goal_usd', 'goal_usd'),  ('backers_count', 'backers_count')]
    list_hue = [('final_status','final_status')]

    plot = scatter_plot(cat_x, cat_y, hue)

    return render_template(
        'scatter.html',
        plot=plot,
        focus_x=cat_x,
        focus_y=cat_y,
        focus_hue=hue,
        drop_x= list_x,
        drop_y= list_y,
        drop_hue= list_hue
    )

##############
## PIE PLOT ##
##############

def pie_plot(hue = 'final_status'):
    


    vcounts = df[hue].value_counts()

    labels = []
    values = []

    for item in vcounts.iteritems():
        labels.append(item[0])
        values.append(item[1])
    
    data = [
        go.Pie(
            labels=labels,
            values=values
        )
    ]

    layout = go.Layout(title='Pie', title_x= 0.48)

    result = {'data': data, 'layout': layout}

    graphJSON = json.dumps(result,cls=plotly.utils.PlotlyJSONEncoder)

    return graphJSON

@app.route('/pie_fn')
def pie_fn():
    hue = request.args.get('hue')

    if hue == None:
        hue = 'final_status'

    list_hue = list_hue = [('final_status','final_status')]

    plot = pie_plot(hue)
    return render_template('pie.html', plot=plot, focus_hue=hue, drop_hue= list_hue)
        
@app.route('/predict')        
def predict():
    ml=pd.read_csv('KS_ML2.csv')
    tab=ml.head(1000)
    return render_template('predict.html',data=tab.values) 
            
        
@app.route('/result', methods=['POST', 'GET'])
def result():
    
    if request.method == 'POST':
    ## Untuk Predict
    
        got=request.form
        
        name=''
        name=got['name']
        name_count=len(name.split())
            
        desc=''
        desc=got['name']
        desc_count=len(desc.split())
            
        backers_count =int(got['backers_count'])
        
        Launched_Deadline =int(got['Launched_Deadline'])
        
        goal_usd =float(got['goal_usd'])

        main_category_Art= ''
        if got['main_category'] == 'Art':
            main_category_Art= 1
        else :
            main_category_Art=0
            
        main_category_Comics= ''
        if got['main_category'] == 'Comics':
            main_category_Comics= 1
        else :
            main_category_Comics =0
            
        main_category_Crafts= ''
        if got['main_category'] == 'Crafts':
            main_category_Crafts= 1
        else :
            main_category_Crafts=0
        
        main_category_Dance=''
        if got['main_category'] == 'Dance':
            main_category_Dance= 1
        else :
            main_category_Dance=0
            
        main_category_Design=''  
        if got['main_category'] == 'Design':
            main_category_Design= 1
        else :
            main_category_Design=0
            
        main_category_Fashion=''  
        if got['main_category'] == 'Fashion':
            main_category_Fashion= 1
        else :
            main_category_Fashion=0
            
        main_category_Film_Video=''   
        if got['main_category'] == 'Film_Video':
            main_category_Film_Video= 1
        else :
            main_category_Film_Video=0
            
        main_category_Food=''
        if got['main_category'] == 'Food':
            main_category_Food= 1
        else :
            main_category_Food=0
            
        main_category_Games=''
        if got['main_category'] == 'Games':
            main_category_Games= 1
        else :
            main_category_Games=0
            
        main_category_Journalism=''   
        if got['main_category'] == 'Journalism':
            main_category_Journalism= 1
        else :
            main_category_Journalism=0
            
        main_category_Music=''
        if got['main_category'] == 'Music':
            main_category_Music= 1
        else :
            main_category_Music=0
            
        main_category_Photography=''
        if got['main_category'] == 'Photography':
            main_category_Photography= 1
        else :
            main_category_Photography=0
            
        main_category_Technology=''
        if got['main_category'] == 'Technology':
            main_category_Technology= 1
        else :
            main_category_Technology=0

        main_category_Publishing=''
        if got['main_category'] == 'Publishing':
            main_category_Publishing= 1
        else :
            main_category_Publishing=0
            
        main_category_Theater=''
        if got['main_category'] == 'Theater':
            main_category_Theater= 1
        else :
            main_category_Theater=0
            
        country_US=''
        if got['country'] == 'US':
            country_US=1
        else:
            country_US=0
        
        country_GB=''
        if got['country'] == 'GB':
            country_GB=1
        else:
            country_GB=0
        
        country_CA=''
        if got['country'] == 'CA':
            country_CA=1
        else:
            country_CA=0
        
        country_AU=''
        if got['country'] == 'AU':
            country_AU=1
        else:
            country_AU=0
        
        country_NL=''
        if got['country'] == 'NL':
            country_NL=1
        else:
            country_NL=0
        
        country_NZ=''
        if got['country'] == 'NZ':
            country_NZ=1
        else:
            country_NZ=0
        
        country_SE=''
        if got['country'] == 'SE':
            country_SE=1
        else:
            country_SE=0
        
        country_DK=''
        if got['country'] == 'DK':
            country_DK=1
        else:
            country_DK=0
            
        country_NO='' 
        if got['country'] == 'NO':
            country_NO=1
        else:
            country_NO=0
        
        country_IE=''
        if got['country'] == 'IE':
            country_IE=1
        else:
            country_IE=0
            
        country_DE=''
        if got['country'] == 'DE':
            country_DE=1
        else:
            country_DE=0
            
            
        currency_USD=''
        if got['currency'] == 'USD':
            currency_USD=1
        else:
            currency_USD=0
        
        currency_GBP=''
        if got['currency'] == 'GBP':
            currency_GBP=1
        else:
            currency_GBP=0
        
        currency_CAD=''
        if got['currency'] == 'CAD':
            currency_CAD=1
        else:
            currency_CAD=0
        
        currency_AUD=''
        if got['currency'] == 'AUD':
            currency_AUD=1
        else:
            currency_AUD=0
        
        currency_EUR=''
        if got['currency'] == 'EUR':
            currency_EUR=1
        else:
            currency_EUR=0
        
        currency_NZD=''
        if got['currency'] == 'NZD':
            currency_NZD=1
        else:
            currency_NZD=0
        
        currency_SEK=''
        if got['currency'] == 'SEK':
            currency_SEK=1
        else:
            currency_SEK=0
        
        currency_DKK=''
        if got['currency'] == 'DKK':
            currency_DKK=1
        else:
            currency_DKK=0
        
        currency_NOK=''
        if got['currency'] == 'NOK':
            currency_NOK=1
        else:
            currency_NOK=0


        Month = ''
        if got['Month'] == 'Jan':
            Month = 1
        elif got['Month'] == 'Feb':
            Month = 2
        elif got['Month'] == 'Mar':
            Month = 3
        elif got['Month'] == 'Apr':
            Month = 4
        elif got['Month'] == 'May':
            Month = 5
        elif got['Month'] == 'Jun':
            Month = 6
        elif got['Month'] == 'Jul':
            Month = 7
        elif got['Month'] == 'Aug':
            Month = 8
        elif got['Month'] == 'Sept':
            Month = 9
        elif got['Month'] == 'Oct':
            Month = 10
        elif got['Month'] == 'Nov':
            Month = 11
        else:
            Month = 12
            

            
        pred = model.predict([[backers_count, Launched_Deadline, goal_usd, Month, name_count,
                               desc_count, main_category_Art, main_category_Comics,
                               main_category_Crafts, main_category_Dance, main_category_Design,
                               main_category_Fashion, main_category_Film_Video,
                               main_category_Food, main_category_Games, main_category_Journalism,
                               main_category_Music, main_category_Photography,
                               main_category_Publishing, main_category_Technology,
                               main_category_Theater, country_AU, country_CA, country_DE,
                               country_DK, country_GB, country_IE, country_NL, country_NO,
                               country_NZ, country_SE, country_US, currency_AUD,
                               currency_CAD, currency_DKK, currency_EUR, currency_GBP,
                               currency_NOK, currency_NZD, currency_SEK, currency_USD]])[0].round(2)

#         ## Untuk Isi Data
        
        main_category_dt= ''
        if got['main_category'] == 'Art':
            main_category_dt= 'Art'
            
        elif got['main_category'] == 'Comics':
            main_category_dt='Comics'

            
        elif got['main_category'] == 'Crafts':
            main_category_dt='Crafts'

            
        elif got['main_category'] == 'Dance':
            main_category_dt='Dance'

            
        elif got['main_category'] == 'Design':
            main_category_dt='Design'
 
            
        elif got['main_category'] == 'Fashion':
            main_category_dt='Fashion'

            
        elif got['main_category'] == 'Film_Video':
            main_category_dt='Film_Video'
 
            
        elif got['main_category'] == 'Food':
            main_category_dt='Food'

            
        elif got['main_category'] == 'Games':
            main_category_dt='Games'

            
        elif got['main_category'] == 'Journalism':
            main_category_dt= 'Journalism'

            
        elif got['main_category'] == 'Music':
            main_category_dt= 'Music'
            
        elif got['main_category'] == 'Photography':
            main_category_dt= 'Photography'

        elif got['main_category'] == 'Technology':
            main_category_dt= 'Technology'
            
        else:
            main_category_dt= 'Theater'
            
            
            
        country_dt= ''
        if got['country'] == 'US':
            country_dt='US'
            
        elif got['country'] == 'GB':
            country_dt='GB'
            
        elif got['country'] == 'CA':
            country_dt='CA'

        elif got['country'] == 'AU':
            country_dt='AU'

        elif got['country'] == 'NL':
            country_dt='NL'

        elif got['country'] == 'NZ':
            country_dt='NZ'
            
        elif got['country'] == 'SE':
            country_dt='SE'

        elif got['country'] == 'DK':
            country_dt='DK'

        elif got['country'] == 'NO':
            country_dt='NO'

        elif got['country'] == 'IE':
            country_dt='IE'

        elif got['country'] == 'DE':
            country_dt='DE'
        
        
        currency_dt= ''
        if got['currency'] == 'USD':
            currency_dt='USD'
            
        elif got['currency'] == 'GBP':
            currency_dt='GBP'
            
        elif got['currency'] == 'CAD':
            currency_dt='CAD'
            
        elif got['currency'] == 'AUD':
            currency_dt='AUD'
            
        elif got['currency'] == 'EUR':
            currency_dt='EUR'
            
        elif got['currency'] == 'NZD':
            currency_dt='NZD'
            
        elif got['currency'] == 'SEK':
            currency_dt='SEK'

        elif got['currency'] == 'DKK':
            currency_dt='DKK'

        else:
            currency_dt='NOK'

        Month_dt= ''
        if got['Month'] == 'Jan':
            Month_dt = 'Jan'
        elif got['Month'] == 'Feb':
            Month_dt = 'Feb'
        elif got['Month'] == 'Mar':
            Month_dt = 'Mar'
        elif got['Month'] == 'Apr':
            Month_dt = 'Apr'
        elif got['Month'] == 'May':
            Month_dt = 'May'
        elif got['Month'] == 'Jun':
            Month_dt = 'Jun'
        elif got['Month'] == 'Jul':
            Month_dt = 'Jul'
        elif got['Month'] == 'Aug':
            Month_dt = 'Aug'
        elif got['Month'] == 'Sept':
            Month_dt = 'Sept'
        elif got['Month'] == 'Oct':
            Month_dt = 'Oct'
        elif got['Month'] == 'Nov':
            Month_dt = 'Nov'
        else:
            Month_dt = 'Des'
            
#         result = [name_count,desc_count,main_category_dt_Art,main_category_dt_Comics,main_category_dt_Crafts,main_category_dt_Dance,
#         main_category_dt_Design,main_category_dt_Fashion,main_category_dt_Film_Video,main_category_dt_Food,main_category_dt_Games,
#         main_category_dt_Journalism,main_category_dt_Music,main_category_dt_Photography,main_category_dt_Publishing,
#         main_category_dt_Technology,main_category_dt_Theater,country_dt_US,country_dt_GB,country_dt_CA,
#         country_dt_AU,country_dt_NL,country_dt_NZ,country_dt_SE,country_dt_DK,country_dt_NO,country_dt_IE,country_dt_DE,Month_dt,Launched_Deadline,
#         backers_count]
        


        return render_template('result.html',
            name=got['name'],desc=got['desc'],backers_count=int(got['backers_count']),
            Launched_Deadline=int(got['Launched_Deadline']),
            goal_usd =float(got['goal_usd']),
            main_category=main_category_dt,
            country=country_dt,
            currency=currency_dt,
            Month=Month_dt,
            ks_pred=pred
                              )

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2020 15:02:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:02:21] "GET /static/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2020 15:02:24] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:02:37] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:02:37] "GET /static/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2020 15:02:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 16:29:28] "GET /cat_fn/False?cat_plot=histplot&cat_x=country&estimator=count&hue=final_status HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 16:29:30] "GET /cat_fn/False?cat_plot=histplot&cat_x=Month&estimator=count&hue=final_status HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 16:29:33] "GET /cat_fn/False?cat_plot=histplot&cat_x=Launched_Deadline&estimator=count&hue=final_status HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 16:29:35] "GET /cat_fn/False?cat_plot=histplot&cat_x=Launched_Deadline&estimator